<a href="https://colab.research.google.com/github/Luna0216/2025-DL-final-project/blob/main/mistral_classifiaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

本程式參考https://medium.com/@lukas.hauzenberger/multilabel-classification-using-mistral-7b-on-a-single-gpu-with-quantization-and-lora-8f848b5237f3
做改寫

In [ ]:
!pip install peft transformers accelerate bitsandbytes datasets

In [ ]:
!pip install dataset

In [ ]:
!pip install scikit-multilearn

In [ ]:
!pip install wandb

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import numpy as np
print(np.__version__)

2.0.2


In [ ]:
import string
import re
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)
def remove_20(text):
  space = re.compile(r'%20')
  return space.sub(r' ', text)

def remove_atsymbol(text):
    name = re.compile(r'@\S+')
    return name.sub(r' atsymbol ', text)


def remove_hashtag(text):
    hashtag = re.compile(r'#')
    return hashtag.sub(r' hashtag ', text)


def remove_exclamation(text):
    exclamation = re.compile(r'!')
    return exclamation.sub(r' exclamation ', text)


def remove_question(text):
    question = re.compile(r'?')
    return question.sub(r' question ', text)


def remove_punc(text):
    return text.translate(str.maketrans('','',string.punctuation))


def remove_number(text):
    number = re.compile(r'\d+')
    return number.sub(r' number ', text)


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               u"\x89û"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' emoji ', string)




In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds)
    }
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}

    d['input_ids'] = [torch.tensor(x) for x in d['input_ids']]
    d['attention_mask'] = [torch.tensor(x) for x in d['attention_mask']]

    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.tensor(d['labels'], dtype=torch.long)

    return d

In [ ]:

import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

In [ ]:
from transformers import Trainer
import torch.nn.functional as F
import torch

class CustomTrainer(Trainer):
    def __init__(self, label_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.label_weights = label_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        if "labels" not in inputs:
            raise ValueError(f"Missing 'labels' key in inputs. Got keys: {inputs.keys()}")

        labels = inputs["labels"]
        inputs = {k: v for k, v in inputs.items() if k != "labels"}

        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fn = torch.nn.CrossEntropyLoss(weight=self.label_weights)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
from torch.utils.data import Dataset

class TorchTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokenized = self.tokenizer(
            self.texts[idx],
            padding=False,
            truncation=True,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in tokenized.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: cherry-tsai0216 (cherry-tsai0216-chang-gung-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

#keyword

In [ ]:
from datasets import Dataset, DatasetDict
class TorchTextDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokenized = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=256,
            return_tensors="pt"
        )
        return {key: val.squeeze(0) for key, val in tokenized.items()}


In [ ]:
import pandas as pd

df = pd.read_csv("train.csv", encoding="utf-8")

In [ ]:
from huggingface_hub import login


HF_TOKEN = ""#這邊的hugging face帳號需要授權mistral模型!
login(token=HF_TOKEN)


In [ ]:
import csv, random
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer


df = pd.read_csv("train.csv")
df = df.sample(frac=1, random_state=0).reset_index(drop=True)


x_train, x_val, y_train, y_val = train_test_split(
    df["text"].tolist(),
    df["target"].tolist(),
    test_size=0.2,
    stratify=df["target"],
    random_state=0
)


'''打開註解即加入外部資料'''
# ptt_df = pd.read_csv("disater_ptt.csv")
# ptt_df['text'] = ptt_df["text_en"]
# ptt_df = ptt_df[ptt_df["text"].str.len() < 60]


#x_train += ptt_df["text"].tolist()
#y_train += ptt_df["target"].astype(int).tolist()

y_train = [int(y) for y in y_train]
y_val = [int(y) for y in y_val]

ds = DatasetDict({
    'train': Dataset.from_dict({'text': x_train, 'labels': y_train}),
    'val': Dataset.from_dict({'text': x_val, 'labels': y_val})
})

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=256)

tokenized_ds = ds.map(preprocess, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [ ]:
ds['train'][1]

{'text': 'Ina Buted Girl Crush??', 'labels': 1}

In [ ]:
#參考https://www.kaggle.com/code/masatoshikato/effective-eda-bert-in-nlp-bert-baseline，清除讓資料剩下純文字
df['text'] = df['text'].apply(lambda text: remove_html(text))
df['text'] = df['text'].apply(lambda text: remove_atsymbol(text))
df['text'] = df['text'].apply(lambda text: remove_hashtag(text))
df['text'] = df['text'].apply(lambda text: remove_exclamation(text))
df['text'] = df['text'].apply(lambda text: remove_punc(text))
df['text'] = df['text'].apply(lambda text: remove_number(text))
df['text'] = df['text'].apply(lambda text: remove_emoji(text))
df['text'] = df['text'].apply(lambda text: remove_20(text))





#參考https://www.kaggle.com/code/divyadevadas/nlp-tweet-bert-roberta-eda-tensorflow-pytorch
df['text'] = df['text'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))



In [ ]:
from huggingface_hub import login


HF_TOKEN = ""
login(token=HF_TOKEN)


In [ ]:
print(ds["val"][15])

{'text': 'A STRONG THUNDERSTORM WILL AFFECT CENTRAL HALIFAX COUNTY THROUGH 1145 PM EDT for Halifax [VA] till 11:45 PM EDT http://t.co/MjSTefgGU5', 'labels': 1}


In [ ]:
import numpy as np
print(np.__version__)

1.26.4


In [ ]:
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


labels = np.asarray(tokenized_ds['train']['labels'])
labels = labels[~pd.isnull(labels)]
labels = labels.astype(int)

classes = np.array([0, 1])


class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=labels)


sample_weights = [class_weights[classes.tolist().index(label)] for label in labels]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)




In [ ]:

from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from transformers import BitsAndBytesConfig
model_name = 'mistralai/Mistral-7B-v0.1'


def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs
HF_TOKEN = ""#因上傳github故拿掉，需要註冊並取得模型使用
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)


lora_config = LoraConfig(
    r = 16,
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = 'SEQ_CLS'
)

# load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    token=HF_TOKEN,
    quantization_config=quantization_config,
    num_labels = len(np.unique(labels))
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

y_train = [int(label) for label in y_train]
y_val = [int(label) for label in y_val]
x_train = [str(t) for t in x_train]
x_val = [str(t) for t in x_val]

In [ ]:
import numpy as np
print(np.__version__)

1.26.4


In [ ]:
print(type(y_train[2]), y_train[:5])

<class 'int'> [1, 1, 0, 0, 0]


In [ ]:
!wandb login
wandb.finish()

wandb: Currently logged in as: cherry-tsai0216 (cherry-tsai0216-chang-gung-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


train/epoch,▁▂▂▃▄▄▅▅▆▇▇█
train/global_step,▁▂▂▃▄▄▅▅▆▇▇█
train/grad_norm,▄▃▄▁█▅▆▃▇▁▅▄
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁
train/loss,▅▆▃▁▃▆▆▁█▁▂▄
train/epoch,0.01515
train/global_step,12
train/grad_norm,105.96889
train/learning_rate,0.0001
train/loss,1.8112


In [ ]:
from transformers import Trainer

def build_collate_fn(tokenizer):
    def collate_fn(batch):
        texts = [item["text"] for item in batch]
        labels = torch.tensor([item["labels"] for item in batch])
        encoding = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
        encoding["labels"] = labels
        return encoding
    return collate_fn

collate_fn = build_collate_fn(tokenizer)


class BalancedBatchTrainer(CustomTrainer):
    def get_train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            sampler=sampler,
            collate_fn=collate_fn
        )

In [ ]:

from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset, DatasetDict
import os
from transformers import EarlyStoppingCallback

os.environ["WANDB_PROJECT"] = "my_binary_project"
wandb.init(name="Binary_Classifier_Run0518_ptt_balanced")


# 訓練參數
training_args = TrainingArguments(
    output_dir="binary_classification",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=1,
    report_to="wandb",
    run_name="Binary_Classifier_Run0518_ptt_balanced"
)


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]

)

# trainer = BalancedBatchTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_ds['train'],
#     eval_dataset=tokenized_ds['val'],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
# )
import json



trainer.train()



<ipython-input-10-23f9de9587af>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `BalancedBatchTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,2.680400,1.417676,0.750492,0.668990,0.777328,0.587156
200,0.829000,0.923155,0.774787,0.758960,0.702211,0.825688
300,0.784500,1.142790,0.796454,0.718182,0.885650,0.603976
400,0.029000,0.725639,0.784636,0.769663,0.711688,0.837920
500,1.317100,0.875602,0.717006,0.732464,0.616510,0.902141
600,0.608100,0.478283,0.830598,0.804545,0.797297,0.811927
700,0.577300,0.469638,0.845043,0.812698,0.844884,0.782875
800,0.331400,0.508564,0.820749,0.797026,0.775687,0.819572
900,0.154300,0.513064,0.837163,0.813814,0.799410,0.828746
1000,0.007500,0.600214,0.854235,0.825746,0.848387,0.804281


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=1100, training_loss=0.686935647680313, metrics={'train_runtime': 12527.8497, 'train_samples_per_second': 1.516, 'train_steps_per_second': 0.19, 'total_flos': 2.001699106770125e+16, 'train_loss': 0.686935647680313, 'epoch': 1.3888888888888888})

In [ ]:
peft_model_id = 'Binary_Classifier_0518_keyword_balanced'
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('Binary_Classifier_0518_keyword_balanced/tokenizer_config.json',
 'Binary_Classifier_0518_keyword_balanced/special_tokens_map.json',
 'Binary_Classifier_0518_keyword_balanced/tokenizer.model',
 'Binary_Classifier_0518_keyword_balanced/added_tokens.json',
 'Binary_Classifier_0518_keyword_balanced/tokenizer.json')

In [ ]:
from huggingface_hub import login, create_repo, upload_folder

HF_TOKEN = ""
HF_USERNAME = ""
REPO_NAME = "binary-classification-mistral_keyword_balanced"
MODEL_DIR = "binary_classification_model_keyword_balanced"


login(token=HF_TOKEN)


create_repo(repo_id=f"{HF_USERNAME}/{REPO_NAME}", repo_type="model", exist_ok=True)


upload_folder(
    repo_id=f"{HF_USERNAME}/{REPO_NAME}",
    folder_path=MODEL_DIR,
    commit_message="Upload fine-tuned binary classification model"
)

print(f" 模型已上傳至：https://huggingface.co/{HF_USERNAME}/{REPO_NAME}")


ValueError: Provided path: '/content/binary_classification_model_keyword_balanced' is not a directory

In [ ]:
from huggingface_hub import login, create_repo, upload_folder
HF_TOKEN = ""#輸入自己的token，要與上面的對應
login(token=HF_TOKEN)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


model_id = ""#輸入自己的model_id，要與上面的對應

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id).to("cuda" if torch.cuda.is_available() else "cpu")


tokenizer_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

##testing

In [ ]:
from torch.utils.data import Dataset

class TestDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {"text": self.texts[idx]}

def build_collate_fn(tokenizer):
    def collate_fn(batch):
        texts = [item["text"] for item in batch]
        return tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    return collate_fn

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader


df_test = pd.read_csv("test.csv")
df_test['text']  = df_test['text'].apply(lambda text: remove_html(text))
df_test['text']  = df_test['text'].apply(lambda text: remove_atsymbol(text))
df_test['text']  = df_test['text'].apply(lambda text: remove_hashtag(text))
df_test['text']  = df_test['text'].apply(lambda text: remove_exclamation(text))
df_test['text']  = df_test['text'].apply(lambda text: remove_punc(text))
df_test['text']  = df_test['text'].apply(lambda text: remove_number(text))
df_test['text']  = df_test['text'].apply(lambda text: remove_emoji(text))
df_test['text']  = df_test['text'].apply(lambda text: remove_20(text))
df_test['text'] = df_test['text'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))




df_test["combined_text"] = [
    f"Keyword: {k.strip() if pd.notna(k) else 'unknown'}\n\nText: {t.strip()}"
    for k, t in zip(df_test["keyword"], df_test["text"])
]
df_test["text"] = df_test["combined_text"].astype(str)

texts = df_test["text"].astype(str).tolist()



test_dataset = TestDataset(texts)
collate_fn = build_collate_fn(tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

In [ ]:
from tqdm import tqdm

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

all_preds = []


with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy().tolist())




Predicting: 100%|██████████| 204/204 [16:09<00:00,  4.75s/it]


In [ ]:
all_preds

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [ ]:
df_test["target"] = all_preds



In [ ]:
df_test[["id","target"]].to_csv('Mistral0518_keyword_pre_balanced_ptt.csv')